In [58]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

#
# SPDX-License-Identifier: GPL-3.0
#
# GNU Radio Python Flow Graph
# Title: Radio Ml Synthetic
# GNU Radio version: 3.10.2.0
# Author David Góez

from packaging.version import Version as StrictVersion

if __name__ == '__main__':
    import ctypes
    import sys
    if sys.platform.startswith('linux'):
        try:
            x11 = ctypes.cdll.LoadLibrary('libX11.so')
            x11.XInitThreads()
        except:
            print("Warning: failed to XInitThreads()")

import numpy as np
import matplotlib.pyplot as plt

from PyQt5 import Qt
from gnuradio import qtgui
from gnuradio.filter import firdes
import sip
from gnuradio import blocks
from gnuradio import gr
from gnuradio.fft import window
import sys
import signal
from argparse import ArgumentParser
from gnuradio.eng_arg import eng_float, intx
from gnuradio import eng_notation

from gnuradio import qtgui

In [76]:
def bin2dec(b):
    number = 0
    for idx, num in enumerate(b[::-1]): 
        number += int(num)*(2**idx)
    return number

def SP(M,sd):
    k=int(np.log2(M))
    bits = np.random.randint(0,2,24)
    # bits = np.array([1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1])
    codebits=[]
    code=[]
    for i in np.arange(0,len(bits),k):
        b="".join(map(str, bits[i:i+k]))
        codebits.append(bin2dec(b))
    for i in codebits:
        code.append(sd[i])
    return k,bits,code

In [79]:

class modulation(gr.sync_block):  # other base classes are basic_block, decim_block, interp_block
    """Embedded Python Block"""

    def __init__(self, M=4):  # only default arguments here
        """arguments to this function show up as parameters in GRC"""
        gr.sync_block.__init__(
            self,
            name='Embedded Modulation',   # will show up in GRC
            in_sig=None,
            out_sig=[np.complex64]
        )
        self.M=M
    def symbols_QAM(self):
        nc=int(self.M/4)
        qam_s=[]
        if self.M==4:dq=1;di=1;sm1=1;sm2=1
        elif self.M==8:dq=1;di=2;sm1=1;sm2=1
        elif self.M==16:dq=2;di=2;sm1=1;sm2=1
        elif self.M==32:dq=3;di=3;sm1=2;sm2=2
        elif self.M==64:dq=5;di=5;sm1=2;sm2=2
        elif self.M==128: dq=6;di=6;sm1=2;sm2=2
        elif self.M==256: dq=7;di=8;sm1=2;sm2=1
        for c in range(1,5):
            cont=0
            for i  in range(1,int(nc/dq)+sm1):   
                for q in range(1,int(nc/di)+sm2):
                    if cont <nc:
                        cont+=1
                        if c==1:
                            qam_s.append(complex(i*2,q*2))
                        elif c==2:
                            qam_s.append(complex(-1*i*2,q*2))
                        elif c==3:
                            qam_s.append(complex(-1*i*2,-1*q*2))
                        elif c==4:
                            qam_s.append(complex(i*2,-1*q*2))
        return qam_s
    
    def work(self, input_items, output_items):
        fs=1.5e5
        fc=20e3
        samples_symbol = 2
        Ts=1/fs

        qam_s=self.symbols_QAM()        
        qam_symbols=np.array(qam_s)

        [k,bits,codebits]=SP(self.M,qam_symbols)
        code = np.repeat(codebits,samples_symbol)
        # print(len(codebits))
        n_symbols = len(codebits)
        duration = n_symbols*(samples_symbol/fs)
        samples_in_duration = n_symbols*samples_symbol
        t = np.linspace(0, duration, num=samples_in_duration)
        code_i = np.real(code)
        code_q = np.imag(code)
        # print(code_i,code_q)
        # qam = code_i*np.cos(2*np.pi*fc*t)-code_q*np.sin(2*np.pi*fc*t)
        qam = code
        tramas = int(len(output_items[0])/len(qam))
        for i in range(0,tramas):
            for x in range(len(qam)):
                output_items[0][x] = qam[x]
        return len(output_items[0])

In [84]:

class Radio_ML_Synthetic(gr.top_block, Qt.QWidget):

    def __init__(self):
        gr.top_block.__init__(self, "Radio Ml Synthetic", catch_exceptions=True)
        Qt.QWidget.__init__(self)
        self.setWindowTitle("Radio Ml Synthetic")
        qtgui.util.check_set_qss()
        try:
            self.setWindowIcon(Qt.QIcon.fromTheme('gnuradio-grc'))
        except:
            pass
        self.top_scroll_layout = Qt.QVBoxLayout()
        self.setLayout(self.top_scroll_layout)
        self.top_scroll = Qt.QScrollArea()
        self.top_scroll.setFrameStyle(Qt.QFrame.NoFrame)
        self.top_scroll_layout.addWidget(self.top_scroll)
        self.top_scroll.setWidgetResizable(True)
        self.top_widget = Qt.QWidget()
        self.top_scroll.setWidget(self.top_widget)
        self.top_layout = Qt.QVBoxLayout(self.top_widget)
        self.top_grid_layout = Qt.QGridLayout()
        self.top_layout.addLayout(self.top_grid_layout)

        self.settings = Qt.QSettings("GNU Radio", "Radio_ML_Synthetic")

        try:
            if StrictVersion(Qt.qVersion()) < StrictVersion("5.0.0"):
                self.restoreGeometry(self.settings.value("geometry").toByteArray())
            else:
                self.restoreGeometry(self.settings.value("geometry"))
        except:
            pass

        ##################################################
        # Variables
        ##################################################
        self.samp_rate = samp_rate = 1e6

        ##################################################
        # Blocks
        ##################################################
        self.qtgui_sink_x_0 = qtgui.sink_c(
            1024, #fftsize
            window.WIN_BLACKMAN_hARRIS, #wintype
            0, #fc
            samp_rate, #bw
            "", #name
            True, #plotfreq
            False, #plotwaterfall
            True, #plottime
            True, #plotconst
            None # parent
        )
        self.qtgui_sink_x_0.set_update_time(1.0/10)
        self._qtgui_sink_x_0_win = sip.wrapinstance(self.qtgui_sink_x_0.qwidget(), Qt.QWidget)

        self.qtgui_sink_x_0.enable_rf_freq(False)

        self.top_layout.addWidget(self._qtgui_sink_x_0_win)
        self.epy_block_0 = modulation(M=256)
        self.blocks_throttle_0 = blocks.throttle(gr.sizeof_gr_complex*1, samp_rate,True)


        ##################################################
        # Connections
        ##################################################
        self.connect((self.blocks_throttle_0, 0), (self.qtgui_sink_x_0, 0))
        self.connect((self.epy_block_0, 0), (self.blocks_throttle_0, 0))


    def closeEvent(self, event):
        self.settings = Qt.QSettings("GNU Radio", "Radio_ML_Synthetic")
        self.settings.setValue("geometry", self.saveGeometry())
        self.stop()
        self.wait()

        event.accept()

    def get_samp_rate(self):
        return self.samp_rate

    def set_samp_rate(self, samp_rate):
        self.samp_rate = samp_rate
        self.blocks_throttle_0.set_sample_rate(self.samp_rate)
        self.qtgui_sink_x_0.set_frequency_range(0, self.samp_rate)




def main(top_block_cls=Radio_ML_Synthetic, options=None):

    if StrictVersion("4.5.0") <= StrictVersion(Qt.qVersion()) < StrictVersion("5.0.0"):
        style = gr.prefs().get_string('qtgui', 'style', 'raster')
        Qt.QApplication.setGraphicsSystem(style)
    qapp = Qt.QApplication(sys.argv)

    tb = top_block_cls()

    tb.start()

    tb.show()

    def sig_handler(sig=None, frame=None):
        tb.stop()
        tb.wait()

        Qt.QApplication.quit()

    signal.signal(signal.SIGINT, sig_handler)
    signal.signal(signal.SIGTERM, sig_handler)

    timer = Qt.QTimer()
    timer.start(500)
    timer.timeout.connect(lambda: None)

    qapp.exec_()

if __name__ == '__main__':
    main()